# email gpt
- input : email draft, receiver and sender
- output : gpt refined email

In [68]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain.embeddings import OllamaEmbeddings
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
# chat model
llm = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    streaing=True
)

In [114]:
# memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,
    memory_key="history",
    return_messages=True
)

In [115]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
# embedding
# embeddings = OllamaEmbeddings(model='mistral:latest')

In [116]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','Generate email using following information. Do not generate too long email. Be concise and polite.'),
        MessagesPlaceholder(variable_name='history'),    
        ('human',"""
        To: {receiver}
        Contents: {contents}
        From: {sender}
        """)
    ]
)

In [117]:
receiver="Prof.K"
sender="Woojin Heo"
contents="hello Professor, I have a question. when can I visit your office?"
print(prompt.format(contents=contents,
                    receiver=receiver,
                    sender=sender,
                    history=[]))

System: Generate email using following information. Do not generate too long email. Be concise and polite.
Human: 
        To: Prof.K
        Contents: hello Professor, I have a question. when can I visit your office?
        From: Woojin Heo
        


In [118]:
def load_memory(_):
    return memory.load_memory_variables({})['history']

In [119]:
# chain
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# invoke with memory
def invoke_chain(contents, receiver, sender):
    result = chain.invoke({'contents':contents,
                        'receiver':receiver,
                        'sender':sender,
                        'history':memory})
    
    memory.save_context({'input':f"'contents':{contents},'receiver':{receiver},'sender':{sender}"},
                    {'output':result.content})
    return result

In [ ]:
# memory.clear() # memory clear
# memory.chat_memory

ChatMessageHistory(messages=[])

In [120]:
# test
contents="hello Professor, I have a question. when can I visit your office?"
receiver='prof.K'
sender='woojin Heo'

result = invoke_chain(contents, receiver, sender)

# result
print(result.content)

 Subject: Request for Office Visit - Woojin Heo

   Dear Professor K,

   I hope this message finds you well. I am writing to inquire about the possibility of visiting your office at a convenient time for you. I have a question regarding our recent discussion and would greatly appreciate your insights.

   Please let me know a suitable date and time that works best for you, and I will make sure to adjust my schedule accordingly.

   Thank you for your time and consideration. I look forward to hearing from you soon.

   Best regards,

   Woojin Heo


In [ ]:
# test
contents="Thank you."
receiver='prof.K'
sender='woojin Heo'

result = invoke_chain(contents, receiver, sender)

# result
print(result.content)

 Subject Re: Request for Office Visit - Woojin Heo

   Dear Professor K,

   Thank you for your prompt response. I appreciate your willingness to meet and discuss my question further.

   I will schedule a visit to your office at the time you suggested. I am looking forward to our meeting and learning more from your expertise.

   Best regards,

   Woojin Heo


In [122]:
# test
contents="I'm busy. let's do it over zoom tmr noon."
receiver='woojin heo'
sender='prof.K'

result = invoke_chain(contents, receiver, sender)

# result
print(result.content)

 Subject Re: Meeting Proposal - Woojin Heo

   Dear Woojin,

   Thank you for your request to meet. Unfortunately, my schedule is quite packed at the moment. I propose we have our meeting over Zoom tomorrow noon instead.

   Please let me know if this time works for you, and I will adjust accordingly.

   Best regards,

   Professor K
